# Heuristics Tests for the Traveling Tournament Problem (TTP)
This notebook tests several heuristics, meta-heuristics, and hybrid methods to solve (usually large) instances of the conventional TTP.

In [3]:
# import libraries

import networkx as nx
import numpy as np
import pandas as pd
import random as rd
import xmltodict, json

### Parse Data

In [5]:
# Convert XML data to JSON

nl4 = xmltodict.parse(open('NL4.xml').read())
nl4 = json.dumps(nl4)
print(nl4)

{"Instance": {"MetaData": {"InstanceName": "NL4", "DataType": "A", "Contributor": "Easton, Nemhauser, and Trick", "Date": {"@day": "0", "@month": "0", "@year": "2001"}, "Country": null, "Remarks": "Based on National Hockey League", "Lowerbound": {"@infeasibility": "0", "@objective": "0"}}, "Structure": {"Format": {"@leagueIds": "0", "numberRoundRobin": "2", "compactness": "C"}, "AdditionalGames": null}, "ObjectiveFunction": {"Objective": "TR"}, "Data": {"Distances": {"distance": [{"@dist": "0", "@team1": "0", "@team2": "0"}, {"@dist": "745", "@team1": "0", "@team2": "1"}, {"@dist": "665", "@team1": "0", "@team2": "2"}, {"@dist": "929", "@team1": "0", "@team2": "3"}, {"@dist": "745", "@team1": "1", "@team2": "0"}, {"@dist": "0", "@team1": "1", "@team2": "1"}, {"@dist": "80", "@team1": "1", "@team2": "2"}, {"@dist": "337", "@team1": "1", "@team2": "3"}, {"@dist": "665", "@team1": "2", "@team2": "0"}, {"@dist": "80", "@team1": "2", "@team2": "1"}, {"@dist": "0", "@team1": "2", "@team2": "

In [ ]:
# Obtain relevant data


### Greedy Algorithm

In [10]:
def greedy_algorithm(n: int) -> dict:
    """
    Greedy algorithm for producing a compact single round-robin schedule for n teams.
    This schedule forms the first half of a double round-robin (DRR) solution.

    Input:
        - n: int --> number of participating teams (must be even for a proper round-robin)
    Output:
        - initial_srr_sol: dict --> initial solution for the single round-robin (SRR), 
                                     where keys are round indices (0 to n-2) and values 
                                     are lists of matchups (tuples of teams).
    """
    num_rounds = n - 1
    initial_srr_sol = {i: [] for i in range(num_rounds)}

    lexographical_matchups = [(i, j) for i in range(n) for j in range(i + 1, n)]
    
    current_round = 0
    while lexographical_matchups:
        matchup = lexographical_matchups.pop(0) 
        team_a, team_b = matchup
        match_placed = False
        
        for _ in range(num_rounds):
            round_index = current_round % num_rounds
        
            
            # Get all teams currently scheduled in the target round
            teams_in_round = set()
            for existing_match in initial_srr_sol[round_index]:
                teams_in_round.add(existing_match[0])
                teams_in_round.add(existing_match[1])

            # Check for clash
            if team_a not in teams_in_round and team_b not in teams_in_round:
                initial_srr_sol[round_index].append(matchup)
                match_placed = True
                current_round = (round_index + 1) % num_rounds
                break
            else:    
                current_round = (current_round + 1) % num_rounds
    #Make the drr
    initial_drr_sol = initial_srr_sol.copy()
    for i in range(num_rounds):
        initial_drr_sol[num_rounds + i] = initial_srr_sol[i]

    return initial_drr_sol


In [11]:
test = greedy_algorithm(10)
print(test)

{0: [(0, 1), (2, 8), (3, 7), (4, 6), (5, 9)], 1: [(0, 2), (1, 9), (3, 8), (4, 7), (5, 6)], 2: [(0, 3), (1, 2), (4, 8), (5, 7), (6, 9)], 3: [(0, 4), (1, 3), (2, 9), (5, 8), (6, 7)], 4: [(0, 5), (1, 4), (2, 3), (6, 8), (7, 9)], 5: [(0, 6), (1, 5), (2, 4), (3, 9), (7, 8)], 6: [(0, 7), (1, 6), (2, 5), (3, 4), (8, 9)], 7: [(0, 8), (1, 7), (2, 6), (3, 5), (4, 9)], 8: [(0, 9), (1, 8), (2, 7), (3, 6), (4, 5)], 9: [(0, 1), (2, 8), (3, 7), (4, 6), (5, 9)], 10: [(0, 2), (1, 9), (3, 8), (4, 7), (5, 6)], 11: [(0, 3), (1, 2), (4, 8), (5, 7), (6, 9)], 12: [(0, 4), (1, 3), (2, 9), (5, 8), (6, 7)], 13: [(0, 5), (1, 4), (2, 3), (6, 8), (7, 9)], 14: [(0, 6), (1, 5), (2, 4), (3, 9), (7, 8)], 15: [(0, 7), (1, 6), (2, 5), (3, 4), (8, 9)], 16: [(0, 8), (1, 7), (2, 6), (3, 5), (4, 9)], 17: [(0, 9), (1, 8), (2, 7), (3, 6), (4, 5)]}


In [2]:
n = 4
lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
print(lexographical_matchups)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


### Simulated Annealing / Tabu Search